In [9]:
import os
import shutil
import numpy as np
import pandas as pd
import datetime
from pathlib import Path
from tqdm import tqdm
from concorde.tools import ascii_replace
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

In [3]:
pathin = Path(r'../data/STORM/processed')
df = pd.read_pickle(pathin/'STORM_NA_R4_In_pnt_v7_MDAvec_12params_1000_map_original_ts_v2.pkl')
dftides = pd.read_pickle(pathin/'STORM_NA_R4_In_pnt_v7_MDAvec_12params_1000_map_original_ts_v2_tide_data.pkl')

In [2]:
pathout = Path(r'../models/adcirc/concorde_NC6/01')
pathtemplate = pathout/'template'

In [18]:
for itc, tc in tqdm(enumerate(df['tc_number'].unique())):
    dfs = df[df['tc_number'] == tc]
    #repeate first row of data for 3 days
    dfr = pd.DataFrame(np.broadcast_to(dfs.iloc[0, :].values, (8*3 + 1, len(dfs.columns))),
                      columns = dfs.columns)
    dates = [dfs['random_date'].iloc[0] - i*pd.Timedelta(hours = 3) for i in range(1, len(dfr)+1)][::-1]
    dfr['random_date'] = dates
    dfr['min_press'] = [1013] * len(dfr)
    dfr['max_ws'] = [1] * len(dfr)
    
    date_aux0 = dfr['random_date'].iloc[0]
    if date_aux0.hour == 0:
        dfr2 = dfr.copy()
    else:
        date_aux1 = datetime.datetime(date_aux0.year, date_aux0.month, date_aux0.day, 0)
        r_extra = pd.date_range(start = str(date_aux1), end = str(date_aux0), freq = '3H')
        dfr_extra = dfr.iloc[0:len(r_extra)-1, :]
        dfr_extra['random_date'] = r_extra[:-1]
        dfr2 = pd.concat([dfr_extra, dfr])
        dfr2.index = range(len(dfr2))
        
    dfe = pd.concat([dfr2, dfs])
    
    try:
        os.mkdir(pathout/f'{itc:03d}')
    except:
        pass
    with open(pathout/f'{itc:03d}'/f'tc_number.{tc}', 'w') as rdm:
        rdm.write(str(tc))
    
    ## write fort22
    with open(pathout/f'{itc:03d}'/'fort.22', 'w') as fout:
        for i in dfe.index:
            d = dfe.loc[i, 'random_date'].strftime('%Y%m%d%H')
            inc = int((dfe.loc[i, 'random_date'] - dfe['random_date'].iloc[0]).total_seconds()/(3600))
            lat = int(np.round(dfe.loc[i, 'lat'], 1)*10)
            lon = int(np.abs(np.round(dfe.loc[i, 'lon'], 1))*10)
            ws = int(np.round(dfe.loc[i, 'max_ws']*1.94384, 0))
            p = int(np.round(dfe.loc[i, 'min_press'], 0))
            rmw = int(np.round(dfe.loc[i, 'rad_to_max_ws'], 0))
            text = (
                    f'AL, 00, {d},   , BEST, {inc:3},{lat:4}N, {lon:4}W, '
                    f'{ws:3}, {p:4},   ,  00, XXX,  000,  000,   00,  000, 0000,     ,'
                    f' {rmw:3},     ,    ,    ,    ,    ,000,  00,  {tc:08d}  ,  00,    0, 0, 0, 0, 0,     00.0,'
                    f'   00.0,   00.0,   00.0,    0.0000,   0.0000,   0.0000,   0.0000,   0.0000,'
                    f' 000.0000, 000.0000, 000.0000, 000.0000\n')
            fout.write(text)

    ## modify fort.15
    filein = pathtemplate/'fort.15'
    pathout_sub = pathout/f'{itc:03d}'
    fileout = pathout_sub/'fort.15'
    olds = ['STORMXX1', 'STORMXX2', 'YYYY_MM_DD_HHmm', 'DDD', 'RRR',
            'M2NODF1', 'M2EQA1', 'S2NODF1', 'S2EQA1', 'N2NODF1', 'N2EQA1', 'K2NODF1', 'K2EQA1', 'K1NODF1', 'K1EQA1', 
            'O1NODF1', 'O1EQA1', 'P1NODF1', 'P1EQA1', 'Q1NODF1', 'Q1EQA1',
            'M2NODF2', 'M2EQA2', 'S2NODF2', 'S2EQA2', 'N2NODF2', 'N2EQA2', 'K2NODF2', 'K2EQA2', 'K1NODF2', 'K1EQA2', 
            'O1NODF2', 'O1EQA2', 'P1NODF2', 'P1EQA2', 'Q1NODF2', 'Q1EQA2',
            'STORMXX3', 'STORMXX4', 'datedate']
    
    dftides_sub = dftides.loc[['M2', 'S2', 'N2', 'K2', 'K1', 'O1', 'P1', 'Q1'], [x for x in dftides.columns if x.endswith(f'_{tc}')]]
    constituents = [f'{x:.5f}' for x in dftides_sub.values.reshape(-1)]
    dur = (dfe['random_date'].iloc[-1] - dfe['random_date'].iloc[0]).total_seconds()/86400
    dramp = (dfr2['random_date'].iloc[-1] - dfr2['random_date'].iloc[0]).total_seconds()/86400
    news = [f'STORM{itc:03d}', f'STORM{itc:03d}', dfe['random_date'].iloc[0].strftime('%Y %m %d %H00'), str(dur), str(dramp)]
    news.extend(constituents)
    news.extend(constituents)
    news.extend([f'STORM{itc:03d}', f'STORM{itc:03d}', str(dfe['random_date'].iloc[0])])
    ascii_replace(filein, fileout, olds, news)
    ## copy
    files_to_copy = ['adcprep', 'adcprep.csh', 'padcirc', 'padcirc.csh', 'fort.13', 'fort.14']
    for f in files_to_copy:
        shutil.copy(pathtemplate/f, pathout_sub/f)

1000it [03:09,  5.27it/s]
